In [10]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

### Step 1: Create a soup object from the home page

In [2]:
url = 'http://chubbygrub.com/'
response = requests.get(url)
print('Status Code: ',response.status_code)

html = response.text

print(html[:700])

Status Code:  200
<!DOCTYPE HTML>
<!--[if lt IE 7 ]> <html lang="en" class="no-js ie6"> <![endif]-->
<!--[if IE 7 ]>    <html lang="en" class="no-js ie7"> <![endif]-->
<!--[if IE 8 ]>    <html lang="en" class="no-js ie8"> <![endif]-->
<!--[if IE 9 ]>    <html lang="en" class="no-js ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html lang="en" class="no-js"> <!--<![endif]-->
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>

  <title>Restaurant Nutrition | Chubbygrub</title>

  <link href='http://fonts.googleapis.com/css?family=Bangers' rel='stylesheet' type='text/css'>
  <link rel="stylesheet" href="http://chubbygrub.com/wp-content/themes/chubbygrub-2/css/bootstrap.css" media="


### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [3]:
soup = BeautifulSoup(html, 'lxml')

In [4]:
restaurants = []

results = soup.find_all('a', {'class' : 'btn btn-lg btn-primary'})

for result in results:
    
    restaurant = {}
    
    restaurant['name'] = result.text
    restaurant['slug'] = result['href'].split('/')[-1]
    
    restaurants.append(restaurant)
    
restaurants

[{'name': 'A&W Restaurants', 'slug': 'aw-restaurants'},
 {'name': "Applebee's", 'slug': 'applebees'},
 {'name': "Arby's", 'slug': 'arbys'},
 {'name': 'Atlanta Bread Company', 'slug': 'atlanta-bread-company'},
 {'name': "Bojangle's Famous Chicken 'n Biscuits",
  'slug': 'bojangles-famous-chicken-n-biscuits'},
 {'name': 'Buffalo Wild Wings', 'slug': 'buffalo-wild-wings'},
 {'name': 'Burger King', 'slug': 'burger-king'},
 {'name': "Captain D's", 'slug': 'captain-ds'},
 {'name': "Carl's Jr.", 'slug': 'carls-jr'},
 {'name': "Charley's Grilled Subs", 'slug': 'charleys-grilled-subs'},
 {'name': 'Chick-fil-A', 'slug': 'chick-fil-a'},
 {'name': "Chili's", 'slug': 'chilis'},
 {'name': 'Chipotle Mexican Grill', 'slug': 'chipotle-mexican-grill'},
 {'name': "Church's", 'slug': 'churchs'},
 {'name': 'Corner Bakery Cafe', 'slug': 'corner-bakery-cafe'},
 {'name': 'Dairy Queen', 'slug': 'dairy-queen'},
 {'name': "Denny's", 'slug': 'dennys'},
 {'name': 'El Pollo Loco', 'slug': 'el-pollo-loco'},
 {'name'

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [7]:
foods = []

In [8]:
for restaurant in restaurants:
        
    res_res = requests.get('http://chubbygrub.com/restaurants/{}'.format(restaurant['slug']))
    res_soup = BeautifulSoup(res_res.content, 'lxml')
    table = res_soup.find('table', {'id': 'items'})
    
    for row in table.find('tbody').find_all('tr'):
        
        food = {}
        
        info = row.find_all('td')
        food['calories'] = info[2].text
        food['carbs'] = info[4].text
        food['category'] = info[1].text
        food['fat'] = info[3].text
        food['name'] = info[0].text
        food['restaurant'] = restaurant['name']
        foods.append(food)

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [12]:
chubby_data = pd.DataFrame(foods)
chubby_data

,calories,carbs,category,fat,name,restaurant
0,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
1,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
2,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
3,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
4,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
5,0,0,\nDrinks\n,0,A&W® Diet Root Beer,A&W Restaurants
6,350,60,\nShakes\n,10,A&W® Diet Root Beer Float (large),A&W Restaurants
7,170,30,\nShakes\n,5,A&W® Diet Root Beer Float (medium),A&W Restaurants
8,170,30,\nShakes\n,5,A&W® Diet Root Beer Float (small),A&W Restaurants
9,600,92,\nShakes\n,18,A&W® Diet Root Beer Freeze (large),A&W Restaurants


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [14]:
chubby_data.to_csv('chubby data')